In [9]:
from Bio import SeqIO
from Bio import Phylo
import random
import os, shutil
from tqdm import tqdm
import subprocess

In [2]:
def generate_n_sequences(number_of_sequences=8000, input_folder="sequences", output_folder="sequences_sampled"):
    os.makedirs(output_folder, exist_ok=True)
    for file in tqdm(os.listdir(input_folder)):
        paresed_seq = list(SeqIO.parse(os.path.join(input_folder, file), format="fasta"))
        sampled_sequences = random.sample(paresed_seq, number_of_sequences)
        SeqIO.write(sampled_sequences, os.path.join(output_folder, file), format="fasta")

In [3]:
generate_n_sequences()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.06it/s]


In [4]:
def aggregate_fasta_files_to_one_file(input_folder="sequences_sampled", output_folder="sequences_aggregated"):
    os.makedirs(output_folder, exist_ok=True)
    with open(os.path.join(output_folder, "sequences_aggregated.fasta"), 'w') as outfile:
        for file in tqdm(os.listdir(input_folder)):
            with open(os.path.join(input_folder, file)) as infile:
                for line in infile:
                    outfile.write(line)

In [5]:
aggregate_fasta_files_to_one_file()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.49it/s]


In [10]:
def run_usearch(input_file="sequences_aggregated/sequences_aggregated.fasta", output_folder="raw_clusters", id=0.9):
    shutil.rmtree(output_folder)
    os.makedirs(output_folder, exist_ok=True)
    cmd = "usearch11.0.667_win32.exe -id {} -cluster_fast {} -clusters {}/".format(id, input_file, output_folder)
    stream = os.popen(cmd)
    print(stream.read())

In [11]:
run_usearch(id=0.1)

usearch v11.0.667_win32, 2.0Gb RAM (8.4Gb total), 8 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: w.celej@student.mini.pw.edu.pl




In [33]:
def filter_clusters(input_folder="raw_clusters", output_folder="filtered_clusters", number_of_species=8):
    os.makedirs(filter_clusters, exist_ok=True)
    selected_clusters = set()
    for file in tqdm(os.listdir(input_folder)):
        seqs_in_cluster = list(SeqIO.parse(os.path.join(input_folder, file), format="fasta"))
        unique_species_ids = set()
        if len(seqs_in_cluster) == number_of_species:
            for seq in seqs_in_cluster:
                seq_name = seq.id.split("|")[-1].split("_")[-1]
                unique_species_ids.add(seq_name)
            if len(unique_species_ids) == number_of_species:
                selected_clusters.add(file)
    return selected_clusters

In [34]:
a = filter_clusters()

100%|██████████████████████████████████████████████████████████████████████████| 14401/14401 [00:02<00:00, 6035.03it/s]


In [35]:
len(a)

1

In [36]:
a

{'8089'}